<b>Scope:</b> Wikipedia Tables Scraper

<b>Delivery:</b> Python Script

<b>Stages:</b> Data exploration, extranction and wranglin. Function development, Script development

<b>Stack:</b> requests, bs4, pandas, bash

<b>Data exploration, extraction and wrangling</b>

In [3]:
#import libraries
import pandas as pd
import requests
import bs4
import numpy as np
import re

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_Nobel_laureates'

In [5]:
response= requests.get(url)
response

<Response [200]>

In [10]:
html= response.content
type(html)

bytes

In [11]:
#pasamos los bytes a etiquetas, lo parseamos con el parseador básico, el html.parser
soup= bs4.BeautifulSoup(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [110]:
#vamos a sacarnos todas las etiquetas que tenemos
parsed_tags= set([tag.name for tag in soup.find_all(True)])
#parsed_tags

In [16]:
tables= soup.find_all('table')
type(tables) #table es una lista pero es de tipo bs4.element.ResultSet

6

In [20]:
element= tables[1]
type(element) # este es un bs4.element.Tag

bs4.element.Tag

In [21]:
# Los mismos métodos aplicados a un tipo bs4.element.Tag hacen cosas distintas que a un bs4.element.ResultSet o un bs4.BeautifulSoup
# por ejemplo table.attr no me devuelve nada pero con element sí me lo devuelve

In [22]:
element.attrs

{'role': 'presentation',
 'class': ['mbox-small', 'plainlinks', 'sistersitebox'],
 'style': 'background-color:#f9f9f9;border:1px solid #aaa;color:#000'}

In [24]:
table= soup.find_all('table',{'class': 'wikitable'} )
len(table)

1

In [30]:
nobels= table[0]
nobels
type (nobels)

bs4.element.Tag

In [60]:
#
nobels.name

'table'

In [61]:
#vemos que nobel tiene estos valores únicos de tags
set ([tag.name for tag in nobels.find_all(True)])

{'a', 'br', 'i', 'span', 'sup', 'tbody', 'td', 'th', 'tr'}

In [41]:

first_row=nobels.find_all('th')
len(first_row)

14

In [42]:
first_row[1]

<th width="18%"><a href="/wiki/List_of_Nobel_laureates_in_Physics" title="List of Nobel laureates in Physics">Physics</a>
</th>

In [50]:
first_row[2].text

'Chemistry\n'

In [62]:
#al hacer esto vemos que los títulos están duplicados
tituloss = [titulo.text.strip() for titulo in first_row]
tituloss

['Year',
 'Physics',
 'Chemistry',
 'Physiologyor Medicine',
 'Literature',
 'Peace',
 'Economics(The Sveriges Riksbank Prize)[13][a]',
 'Year',
 'Physics',
 'Chemistry',
 'Physiologyor Medicine',
 'Literature',
 'Peace',
 'Economics']

In [68]:
titulos= []
for t in range(int(len(first_row)/2)):
    titulos.append(first_row[t].text)
titulos

['Year\n',
 'Physics\n',
 'Chemistry\n',
 'Physiologyor Medicine\n',
 'Literature\n',
 'Peace\n',
 'Economics(The Sveriges Riksbank Prize)[13][a]\n']

In [69]:
#para limpiar vamos a usar regex
string= 'Economics(The Sveriges Riksbank Prize)[13][a]\n'

In [70]:
#una opción sólo para este ejemplo
string.split('[')[0]

'Economics(The Sveriges Riksbank Prize)'

In [74]:
#sacamos los títulos limpios
clean_titles= []
for s in titulos:
    clean_titles.append(s.replace('\n','').split('[')[0])
    
clean_titles

['Year',
 'Physics',
 'Chemistry',
 'Physiologyor Medicine',
 'Literature',
 'Peace',
 'Economics(The Sveriges Riksbank Prize)']

In [76]:
#Ahora vamos a por las filas
rows= nobels.find_all('tr')
len(rows)

123

In [106]:
#chequeo y veo que la row 0 no la voy a querer
row= rows[0]



In [108]:
elements= [i.text.strip() for i in row][1::2]
elements

['Year',
 'Physics',
 'Chemistry',
 'Physiologyor Medicine',
 'Literature',
 'Peace',
 'Economics(The Sveriges Riksbank Prize)[13][a]']

In [117]:
datos= [i.text.replace('\n','') for row in rows for i in row][1::2]
datos

['Year',
 'Physics',
 'Chemistry',
 'Physiologyor Medicine',
 'Literature',
 'Peace',
 'Economics(The Sveriges Riksbank Prize)[13][a]',
 '1901',
 'Wilhelm Röntgen',
 "Jacobus Henricus van 't Hoff",
 'Emil Adolf von Behring',
 'Sully Prudhomme',
 'Henry Dunant;Frédéric Passy',
 '—',
 '1902',
 'Hendrik Lorentz;Pieter Zeeman',
 'Hermann Emil Fischer',
 'Ronald Ross',
 'Theodor Mommsen',
 'Élie Ducommun;Charles Albert Gobat',
 '—',
 '1903',
 'Henri Becquerel;Pierre Curie;Marie Curie',
 'Svante Arrhenius',
 'Niels Ryberg Finsen',
 'Bjørnstjerne Bjørnson',
 'Randal Cremer',
 '—',
 '1904',
 'Lord Rayleigh',
 'William Ramsay',
 'Ivan Pavlov',
 'Frédéric Mistral;José Echegaray',
 'Institut de Droit International',
 '—',
 '1905',
 'Philipp Lenard',
 'Adolf von Baeyer',
 'Robert Koch',
 'Henryk Sienkiewicz',
 'Bertha von Suttner',
 '—',
 '1906',
 'J. J. Thomson',
 'Henri Moissan',
 'Camillo Golgi;Santiago Ramón y Cajal',
 'Giosuè Carducci',
 'Theodore Roosevelt',
 '—',
 '1907',
 'Albert Abraham M

In [123]:
# Probamos con numpy cómo conseguir nuestro DataFrame
x= len (datos)
y= int(len (clean_titles))

shape=int(x/y)
shape

121

In [126]:
array=np.array(datos)
array= array.reshape(shape, y)
array

array([['Year', 'Physics', 'Chemistry', 'Physiologyor Medicine',
        'Literature', 'Peace',
        'Economics(The Sveriges Riksbank Prize)[13][a]'],
       ['1901', 'Wilhelm Röntgen', "Jacobus Henricus van 't Hoff",
        'Emil Adolf von Behring', 'Sully Prudhomme',
        'Henry Dunant;Frédéric Passy', '—'],
       ['1902', 'Hendrik Lorentz;Pieter Zeeman', 'Hermann Emil Fischer',
        'Ronald Ross', 'Theodor Mommsen',
        'Élie Ducommun;Charles Albert Gobat', '—'],
       ['1903', 'Henri Becquerel;Pierre Curie;Marie Curie',
        'Svante Arrhenius', 'Niels Ryberg Finsen',
        'Bjørnstjerne Bjørnson', 'Randal Cremer', '—'],
       ['1904', 'Lord Rayleigh', 'William Ramsay', 'Ivan Pavlov',
        'Frédéric Mistral;José Echegaray',
        'Institut de Droit International', '—'],
       ['1905', 'Philipp Lenard', 'Adolf von Baeyer', 'Robert Koch',
        'Henryk Sienkiewicz', 'Bertha von Suttner', '—'],
       ['1906', 'J. J. Thomson', 'Henri Moissan',
        'Cam

In [139]:
# con el drop quitamos la primera fila que es igual que el índice y luego un reset index
df= pd.DataFrame(array, columns= clean_titles).drop([0]).reset_index(drop=True)
df

,Year,Physics,Chemistry,Physiologyor Medicine,Literature,Peace,Economics(The Sveriges Riksbank Prize)
0,1901,Wilhelm Röntgen,Jacobus Henricus van 't Hoff,Emil Adolf von Behring,Sully Prudhomme,Henry Dunant;Frédéric Passy,—
1,1902,Hendrik Lorentz;Pieter Zeeman,Hermann Emil Fischer,Ronald Ross,Theodor Mommsen,Élie Ducommun;Charles Albert Gobat,—
2,1903,Henri Becquerel;Pierre Curie;Marie Curie,Svante Arrhenius,Niels Ryberg Finsen,Bjørnstjerne Bjørnson,Randal Cremer,—
3,1904,Lord Rayleigh,William Ramsay,Ivan Pavlov,Frédéric Mistral;José Echegaray,Institut de Droit International,—
4,1905,Philipp Lenard,Adolf von Baeyer,Robert Koch,Henryk Sienkiewicz,Bertha von Suttner,—
...,...,...,...,...,...,...,...
115,2018,Arthur Ashkin;Gérard Mourou;Donna Strickland,Frances H. Arnold;George P. Smith;Greg Winter,James P. Allison; Tasuku Honjo,Olga Tokarczuk [h],Denis Mukwege; Nadia Murad,William Nordhaus; Paul Romer
116,2019,James Peebles; Michel Mayor; Didier Queloz,John B. Goodenough;M. Stanley Whittingham;Akir...,William Kaelin Jr.;Peter J. Ratcliffe;Gregg L....,Peter Handke,Abiy Ahmed,Abhijit Banerjee;Esther Duflo;Michael Kremer
117,2020,Roger Penrose; Reinhard Genzel; Andrea M. Ghez,Emmanuelle Charpentier;Jennifer Doudna,Harvey J. Alter; Michael Houghton; Charles M. ...,Louise Glück,World Food Programme,Paul R. Milgrom;Robert B. Wilson
118,2021,Syukuro Manabe; Klaus Hasselmann; Giorgio Parisi,Benjamin List; David MacMillan,David Julius; Ardem Patapoutian,Abdulrazak Gurnah,Maria Ressa; Dmitry Muratov,David Card;Joshua D. Angrist;Guido W. Imbens


In [119]:
df.to_csv()

7